In [48]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader

# Load dataset
data = fetch_california_housing()
X, y = data.data, data.target

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# DataLoader
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)


In [49]:
import torch.nn as nn

class RegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(X_train.shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.linear(x)

model = RegressionModel()


In [50]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-8)

In [51]:
for epoch in range(100):
    model.train()
    for X, y in train_loader:
        preds = model(X)
        loss = loss_fn(preds, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")


Epoch 0, Loss: 0.7384
Epoch 10, Loss: 0.2425
Epoch 20, Loss: 0.6569
Epoch 30, Loss: 0.2239
Epoch 40, Loss: 0.2626
Epoch 50, Loss: 0.2721
Epoch 60, Loss: 0.2990
Epoch 70, Loss: 0.2685
Epoch 80, Loss: 0.2584
Epoch 90, Loss: 0.3353


In [52]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = loss_fn(predictions, y_test)
    print(f"Test MSE: {test_loss.item():.4f}")


Test MSE: 0.3062
